标准库中最值得关注的两个装饰器是lru_cache和全新的singledispatch

lru_cache用来把耗时的函数的结果保存起来，避免传入相同的参数时重复计算。
sigledispatsh用来为不同的传参类型实现不同的函数

### 7.8.1 使用functools.lru_cache做备忘

In [1]:
# functools.lru_cache是非常实用的装饰器，它实现了备忘（memoization）功能。
# 这是一项优化技术，它把耗时的函数的结果保存起来，避免传入相同的参数时重复计算。

from scripts.clockdeco import clock

# 没有使用优化器的版本
@clock
def fibnonacci(n):
    if n < 2:
        return n
    return fibnonacci(n - 1) + fibnonacci(n - 2)

print(fibnonacci(6))

[0.00000000s]fibnonacci(1)->1 
[0.00000000s]fibnonacci(0)->0 
[0.00000000s]fibnonacci(2)->1 
[0.00000000s]fibnonacci(1)->1 
[0.00000000s]fibnonacci(3)->2 
[0.00000000s]fibnonacci(1)->1 
[0.00000000s]fibnonacci(0)->0 
[0.00000000s]fibnonacci(2)->1 
[0.00000000s]fibnonacci(4)->3 
[0.00000000s]fibnonacci(1)->1 
[0.00000000s]fibnonacci(0)->0 
[0.00000000s]fibnonacci(2)->1 
[0.00000000s]fibnonacci(1)->1 
[0.00000000s]fibnonacci(3)->2 
[0.00000000s]fibnonacci(5)->5 
[0.00000000s]fibnonacci(1)->1 
[0.00000000s]fibnonacci(0)->0 
[0.00000000s]fibnonacci(2)->1 
[0.00000000s]fibnonacci(1)->1 
[0.00000000s]fibnonacci(3)->2 
[0.00000000s]fibnonacci(1)->1 
[0.00000000s]fibnonacci(0)->0 
[0.00000000s]fibnonacci(2)->1 
[0.00000000s]fibnonacci(4)->3 
[0.00000000s]fibnonacci(6)->8 
8


In [6]:
# 使用缓存实现优化

import functools
from scripts.clockdeco import clock

# 这里的括号说明这个deco可以传入参数
# 叠放装饰器说明将clock返回的函数传入lru中 【就近原则】
@functools.lru_cache()
@clock
def fibnonacci(n):
    return n if n < 2 else fibnonacci(n-1)+fibnonacci(n-2)

print(fibnonacci(30))

[0.00000000s]fibnonacci(1)->1 
[0.00000000s]fibnonacci(0)->0 
[0.00099802s]fibnonacci(2)->1 
[0.00099802s]fibnonacci(3)->2 
[0.00099802s]fibnonacci(4)->3 
[0.00099802s]fibnonacci(5)->5 
[0.00099802s]fibnonacci(6)->8 
[0.00099802s]fibnonacci(7)->13 
[0.00099802s]fibnonacci(8)->21 
[0.00099802s]fibnonacci(9)->34 
[0.00099802s]fibnonacci(10)->55 
[0.00099802s]fibnonacci(11)->89 
[0.00099802s]fibnonacci(12)->144 
[0.00099802s]fibnonacci(13)->233 
[0.00099802s]fibnonacci(14)->377 
[0.00099802s]fibnonacci(15)->610 
[0.00099802s]fibnonacci(16)->987 
[0.00099802s]fibnonacci(17)->1597 
[0.00099802s]fibnonacci(18)->2584 
[0.00099802s]fibnonacci(19)->4181 
[0.00099802s]fibnonacci(20)->6765 
[0.00099802s]fibnonacci(21)->10946 
[0.00099802s]fibnonacci(22)->17711 
[0.00099802s]fibnonacci(23)->28657 
[0.00099802s]fibnonacci(24)->46368 
[0.00099802s]fibnonacci(25)->75025 
[0.00099802s]fibnonacci(26)->121393 
[0.00099802s]fibnonacci(27)->196418 
[0.00099802s]fibnonacci(28)->317811 
[0.00099802s]fibnona

### 7.8.2 单分派泛函数

In [7]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}<pre>'.format(content)

In [8]:
# 使用单分派泛函数实现

from functools import singledispatch
from collections import abc
import numbers
import html

# singledispath标记处理obj类型的基函数
@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}<pre>'.format(content)
# 各个专门函数用 @[base_function].register([type])装饰
@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)
@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)
# 可以叠放多个装饰器让同一个函数支持不同的类型
@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>'+inner+'</li>\n</ul>'

<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]<pre>
